In [1]:
import os, sys

##### Toy Example to create inode sort-discepancy through compression

In [40]:
cmd = '''
mkdir demo-dogs/dogs
touch demo-dogs/dogs/dog_1.txt
touch demo-dogs/dogs/dog_3.txt
touch demo-dogs/dogs/dog_2.txt
ls -i demo-dogs/dogs
tar -cf demo-dogs/dogs.tar demo-dogs/dogs --sort=name
mkdir demo-dogs/new-dogs
tar -xf demo-dogs/dogs.tar -C demo-dogs/new-dogs/
ls -i demo-dogs/new-dogs/demo-dogs/dogs
rm demo-dogs/new-dogs -r
rm demo-dogs/dogs -r
'''

s_cmd = ' && '.join(cmd.split('\n')[1:-1])
# s_cmd

! $s_cmd

4325397 dog_1.txt  4325399 dog_2.txt  4325398 dog_3.txt
4325404 dog_1.txt  4325405 dog_2.txt  4325406 dog_3.txt


From ls printout you can see inode is *not* in alphanumeric order for original direcotry (because the order of creation) but after compression/decompression, the inode *is* in alphanumeric order in the second directory.

##### Screenshots of the inode listing for different machines